In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob 
from matplotlib.pyplot import cm

def set_fig():     
    
    plt.rc('xtick', labelsize=40) 
    plt.rc('ytick', labelsize=40) 
    plt.rc('axes', labelsize=60) # fontsize of the x and y labels
    
#    ax.xaxis.set_minor_locator(AutoMinorLocator(2))  
#    ax.yaxis.set_minor_locator(AutoMinorLocator(2))

    ax.tick_params(which='major', bottom=True, top=True, left=True, right=True, length=15, width=3, direction="in", pad=20)
    ax.tick_params(which='minor', bottom=True, top=True, left=True, right=True, length=10, width=3, direction="in", pad=20)
    
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(3)
        
plt.rcParams['figure.figsize'] = [10, 8]

K_to_kJ = 1.38064878066852E-26
avogadro = 6.02214E23

### Finding & Sorting Files

In [ ]:
files = []
for file in glob.glob("*pot*"):
    if "multi" in file:
        print("ignoring notebook")
    else:
        files.append(file)

def sortfile(files):
    return int(files[0].split("_")[0])
files.sort(key=sortfile)
files = sorted(files, key=lambda x: int(x.split('_')[0]))
f_pot = files
print(f_pot)

files = []
for file in glob.glob("*nads*"):
    if "multi" in file:
        print("ignoring notebook")
    else:
        files.append(file)

def sortfile(files):
    return int(files[0].split("_")[0])
files.sort(key=sortfile)
files = sorted(files, key=lambda x: int(x.split('_')[0]))
f_nads = files
print(f_nads)

files = []
for file in glob.glob("*ha*"):
    if "multi" in file:
        print("ignoring notebook")
    else:
        files.append(file)

def sortfile(files):
    return int(files[0].split("_")[0])
files.sort(key=sortfile)
files = sorted(files, key=lambda x: int(x.split('_')[0]))
f_ha = files
print(f_ha)

files = []
for file in glob.glob("*aa*"):
    if "multi" in file:
        print("ignoring notebook")
    else:
        files.append(file)

def sortfile(files):
    return int(files[0].split("_")[0])
files.sort(key=sortfile)
files = sorted(files, key=lambda x: int(x.split('_')[0]))
f_aa = files
print(f_aa)

nsteps = len(f_aa)
print(nsteps)

pressures = ["1", "5", "1e1", "5e1", "1e2", "5e2", "1e3", "5e3", "1e4", "5e4", "1e5", "5e5", "1e6", "5e6", "1e7", "5e7", "1e8"]
press = []
for i in range(len(pressures)):
    a = float(pressures[i])
    press.append(a)

### Reading & Processing Files 

In [ ]:
ener = []
ener_ave = []
for file in f_pot:
    with open(file, "r") as f:
        e = [] 
        a = []
        for i, line in enumerate(f):
            line = line.split()
            e.append(float(line[4]))
            if i >= 250:
                c = line[7]
                c = c[:-1]
                a.append(float(c))
    data = np.array(e)
    ave = np.array(a)

    ener.append(data)
    ener_ave.append(ave)

ave_e = []
std_e = []

for a in ener:
    ave_e.append(np.average(a))
    std_e.append(np.std(a))

aa = []
aa_ave = []
for file in f_aa:
    with open(file, "r") as f:
        e = [] 
        a = []
        for i, line in enumerate(f):
            line = line.split()
            e.append(float(line[3]))
            if i >= 500:
                c = line[6]
                c = c[:-1]
                a.append(float(c))
    data = np.array(e)
    ave = np.array(a)

    aa.append(data)
    aa_ave.append(ave)

ave_aa = []
std_aa = []
for a in aa:
    ave_aa.append(np.average(a))
    std_aa.append(np.std(a))

ha = []
ha_ave = []
for file in f_ha:
    with open(file, "r") as f:
        e = [] 
        a = []
        for i, line in enumerate(f):
            line = line.split()
            e.append(float(line[3]))
            if i >= 500:
                c = line[6]
                c = c[:-1]
                a.append(float(c))
    data = np.array(e)
    ave = np.array(a)

    ha.append(data)
    ha_ave.append(ave)

ave_ha = []
std_ha = [] 
for a in ha:
    ave_ha.append(np.average(a))
    std_ha.append(np.std(a))

na = []
for file in f_nads:
    with open(file, "r") as f:
        e = [] 
        a = []
        for i, line in enumerate(f):
            line = line.split()
            e.append(float(line[3]))
    data = np.array(e)

    na.append(data)

ave_na = []
std_na = []
for a in na:
    ave_na.append(np.average(a))
    std_na.append(np.std(a))

mpa = [] 
for i in press:
    mpa.append(i/1000000)


### Checking Convergence

In [ ]:
# Sliding window with varied window sizes 
window_sizes = np.arange(50, 550, 50)

# Find max length traj 
shortest = 0 
for ix in range(len(press)):
    if ix == 0 :
        shortest = len(na[ix])
    else:
        shortest = min(shortest, len(na[ix]))


all_starting_points = []

for ix in window_sizes:
    starting_points = np.arange(0, shortest-(1+ix), 20)
    all_starting_points.append(starting_points)

all_hs = []
all_dhs = [] 
a_variable = 0

for starting_points in all_starting_points:
    window_size = window_sizes[a_variable]
    hs = [] 
    dhs = [] 

    for start in starting_points:
        q_calc_kj = np.zeros((nsteps,4))
        q_calc_kj_err = np.zeros((nsteps,4))
        ave_n2 = [] 

        for ix in range(nsteps):
            a = []
            n2 = []
            lists = np.arange(start, window_size+start, 1)

            for iy in lists:
                a.append(na[ix][iy]*ener[ix][iy]*K_to_kJ)
                n2.append((na[ix][iy])**2)

            q_calc_kj[ix,0] = np.average(a)
            q_calc_kj[ix,1] = np.average(ener[ix][start:window_size+start])*K_to_kJ
            q_calc_kj[ix,2] = np.average(na[ix][start:window_size+start])
            q_calc_kj[ix,3] = np.var(na[ix][start:window_size+start])

            q_calc_kj_err[ix,0] = np.std(a)
            q_calc_kj_err[ix,1] = np.std(ener[ix][start:window_size+start])*K_to_kJ
            q_calc_kj_err[ix,2] = np.std(na[ix][start:window_size+start])
            q_calc_kj_err[ix,3] = np.std(n2)

            ave_n2.append(np.average(n2))


        all_tot = []
        all_dtot = [] 


        for ix in range(len(press)):
            a = q_calc_kj[ix,0]
            b = q_calc_kj[ix,1]
            c = q_calc_kj[ix,2]
            d = ave_n2[ix]
            e = q_calc_kj[ix,2]
            f = q_calc_kj[ix,2]

            da = q_calc_kj_err[ix,0]
            db = q_calc_kj_err[ix,1]
            dc = q_calc_kj_err[ix,2]
            dd = q_calc_kj_err[ix,3]
            de = q_calc_kj_err[ix,2]
            df = q_calc_kj_err[ix,2]

            bc = b*c 
            dbc = bc*(np.sqrt((db/b)**2 + (dc/c)**2))

            a_bc = a - bc 
            da_bc = np.sqrt((da)**2)+((dbc)**2)

            ###################################
            ef = e*f 
            Def = ef*(np.sqrt((de/e)**2 + (df/f)**2))
            
            d_ef = d - ef
            dd_ef = np.sqrt(dd**2 + Def**2)

            ########################################

            tot = a_bc/d_ef
            dtot = tot*(np.sqrt((da_bc/a_bc)**2)+ (dd_ef/d_ef)**2)

            tot_avo = tot*avogadro
            dtot_avo = tot_avo*(np.sqrt((tot/dtot)**2) + 0)

            tot_avo_RT = R*T - tot_avo
            dtot_avo_RT = np.sqrt(dtot_avo**2)
            
            all_tot.append(tot_avo_RT)
            all_dtot.append(abs(dtot))

        hs.append(all_tot)
        dhs.append(all_dtot)

    all_hs.append(hs)
    all_dhs.append(dhs)
    a_variable = a_variable + 1



In [ ]:
#quant = np.arange(7,13,1)
plt.rcParams['figure.figsize'] = [25, 30]

fig, ax = plt.subplots(4,2)
outer_grid = fig.add_gridspec(4, 4, wspace=0, hspace=0)

# Row 1  
sp = 0 
ax[0,0].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[0,0].set_xlabel("Pressure (MPa)", fontsize=15)
ax[0,0].set_ylabel(r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)

color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[0,0].scatter(mpa[:], all_hs[sp][ix][:], color=c, s=50)
            ax[0,0].plot(mpa[:], all_hs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

sp = 1
ax[0,1].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[0,1].set_xlabel("Pressure (MPa)", fontsize=15)
ax[0,1].set_ylabel(r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[0,1].scatter(mpa[:], all_hs[sp][ix][:], color=c, s=50)
            ax[0,1].plot(mpa[:], all_hs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

# Row 2
sp = 2
ax[1,0].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[1,0].set_xlabel("Pressure (MPa)", fontsize=15)
ax[1,0].set_ylabel(r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[1,0].scatter(mpa[:], all_hs[sp][ix][:], color=c, s=50)
            ax[1,0].plot(mpa[:], all_hs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

sp = 3
ax[1,1].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[1,1].set_xlabel("Pressure (MPa)", fontsize=15)
ax[1,1].set_ylabel(r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[1,1].scatter(mpa[:], all_hs[sp][ix][:], color=c, s=50)
            ax[1,1].plot(mpa[:], all_hs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

# Row 3
sp = 4
ax[2,0].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[2,0].set_xlabel("Pressure (MPa)", fontsize=15)
ax[2,0].set_ylabel(r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[2,0].scatter(mpa[:], all_hs[sp][ix][:], color=c, s=50)
            ax[2,0].plot(mpa[:], all_hs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)
sp = 5
ax[2,1].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[2,1].set_xlabel("Pressure (MPa)", fontsize=15)
ax[2,1].set_ylabel(r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[2,1].scatter(mpa[:], all_hs[sp][ix][:], color=c, s=50)
            ax[2,1].plot(mpa[:], all_hs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)
# Row 4 
sp = 6
ax[3,0].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[3,0].set_xlabel("Pressure (MPa)", fontsize=15)
ax[3,0].set_ylabel(r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[3,0].scatter(mpa[:], all_hs[sp][ix][:], color=c, s=50)
            ax[3,0].plot(mpa[:], all_hs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

sp = 7
ax[3,1].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[3,1].set_xlabel("Pressure (MPa)", fontsize=15)
ax[3,1].set_ylabel(r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[3,1].scatter(mpa[:], all_hs[sp][ix][:], color=c, s=50)
            ax[3,1].plot(mpa[:], all_hs[sp][ix][:], color=c, label="Starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

ax[0,0].set_xscale('log')
ax[1,0].set_xscale('log')
ax[2,0].set_xscale('log')
ax[3,0].set_xscale('log')
ax[0,1].set_xscale('log')
ax[1,1].set_xscale('log')
ax[2,1].set_xscale('log')
ax[3,1].set_xscale('log')

ax[3,1].legend(loc='upper center', bbox_to_anchor=(-0.17, -0.05),fancybox=False, shadow=True, ncol=6, fontsize=15)


In [ ]:
plt.rcParams['figure.figsize'] = [25, 30]

fig, ax = plt.subplots(4,2)
outer_grid = fig.add_gridspec(4, 4, wspace=0, hspace=0)

# Row 1  
sp = 0 
ax[0,0].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[0,0].set_xlabel("Pressure (MPa)", fontsize=15)
ax[0,0].set_ylabel("Uncertainties in "+ r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)

color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[0,0].scatter(mpa[:], all_dhs[sp][ix][:], color=c, s=50)
            ax[0,0].plot(mpa[:], all_dhs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

sp = 1
ax[0,1].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[0,1].set_xlabel("Pressure (MPa)", fontsize=15)
ax[0,1].set_ylabel("Uncertainties in "+r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[0,1].scatter(mpa[:], all_dhs[sp][ix][:], color=c, s=50)
            ax[0,1].plot(mpa[:], all_dhs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

# Row 2
sp = 2
ax[1,0].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[1,0].set_xlabel("Pressure (MPa)", fontsize=15)
ax[1,0].set_ylabel("Uncertainties in "+r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[1,0].scatter(mpa[:], all_dhs[sp][ix][:], color=c, s=50)
            ax[1,0].plot(mpa[:], all_dhs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

sp = 3
ax[1,1].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[1,1].set_xlabel("Pressure (MPa)", fontsize=15)
ax[1,1].set_ylabel("Uncertainties in "+r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[1,1].scatter(mpa[:], all_dhs[sp][ix][:], color=c, s=50)
            ax[1,1].plot(mpa[:], all_dhs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

# Row 3
sp = 4
ax[2,0].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[2,0].set_xlabel("Pressure (MPa)", fontsize=15)
ax[2,0].set_ylabel("Uncertainties in "+r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[2,0].scatter(mpa[:], all_dhs[sp][ix][:], color=c, s=50)
            ax[2,0].plot(mpa[:], all_dhs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)
sp = 5
ax[2,1].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[2,1].set_xlabel("Pressure (MPa)", fontsize=15)
ax[2,1].set_ylabel("Uncertainties in "+r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[2,1].scatter(mpa[:], all_dhs[sp][ix][:], color=c, s=50)
            ax[2,1].plot(mpa[:], all_dhs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)
# Row 4 
sp = 6
ax[3,0].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[3,0].set_xlabel("Pressure (MPa)", fontsize=15)
ax[3,0].set_ylabel("Uncertainties in "+r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[3,0].scatter(mpa[:], all_dhs[sp][ix][:], color=c, s=50)
            ax[3,0].plot(mpa[:], all_dhs[sp][ix][:], color=c, label="starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

sp = 7
ax[3,1].set_title("Window Size = " + str(window_sizes[sp]), fontsize=20 )
ax[3,1].set_xlabel("Pressure (MPa)", fontsize=15)
ax[3,1].set_ylabel("Uncertainties in "+r"$H_{\mathrm{ads}}$ (kJ/mol)", fontsize=15)
color = iter(cm.rainbow(np.linspace(0, 1, len(all_starting_points[sp]))))
for ix in range(len(all_starting_points[sp])):
        c = next(color)
        if ix != 0:
            ax[3,1].scatter(mpa[:], all_dhs[sp][ix][:], color=c, s=50)
            ax[3,1].plot(mpa[:], all_dhs[sp][ix][:], color=c, label="Starting_point="+str(all_starting_points[sp][ix]), linewidth=1)

ax[0,0].set_xscale('log')
ax[1,0].set_xscale('log')
ax[2,0].set_xscale('log')
ax[3,0].set_xscale('log')
ax[0,1].set_xscale('log')
ax[1,1].set_xscale('log')
ax[2,1].set_xscale('log')
ax[3,1].set_xscale('log')

#ax[3,1].legend(loc='upper center', bbox_to_anchor=(-0.17, -0.05),fancybox=True, shadow=True, ncol=6, fontsize=15)


### Calculating isosteric heat of adsorption 

In [ ]:
# Do this with conversion of energy from K to kJ

last_steps = 50
q_calc_kj = np.zeros((nsteps,4))
q_calc_kj_err = np.zeros((nsteps,4))
ave_n2 = [] 

for ix in range(nsteps):
    a = []
    n2 = []
    list = np.arange(len(na[ix])-last_steps, len(na[ix]), 1)
    for iy in list:
        a.append(na[ix][iy]*ener[ix][iy]*K_to_kJ)
        n2.append((na[ix][iy])**2)

    q_calc_kj[ix,0] = np.average(a)
    q_calc_kj[ix,1] = np.average(ener[ix][-last_steps:])*K_to_kJ
    q_calc_kj[ix,2] = np.average(na[ix][-last_steps:])
    q_calc_kj[ix,3] = np.var(na[ix][-last_steps:])

    q_calc_kj_err[ix,0] = np.std(a)
    q_calc_kj_err[ix,1] = np.std(ener[ix][-last_steps:])*K_to_kJ
    q_calc_kj_err[ix,2] = np.std(na[ix][-last_steps:])
    q_calc_kj_err[ix,3] = np.std(n2)

    ave_n2.append(np.average(n2))

all_tot = []
all_dtot = [] 

R = 8.3145E-3
T = 298
avogadro = 6.02214E23

for ix in range(len(press)):
    a = q_calc_kj[ix,0]
    b = q_calc_kj[ix,1]
    c = q_calc_kj[ix,2]
    d = ave_n2[ix]
    e = q_calc_kj[ix,2]
    f = q_calc_kj[ix,2]

    da = q_calc_kj_err[ix,0]
    db = q_calc_kj_err[ix,1]
    dc = q_calc_kj_err[ix,2]
    dd = q_calc_kj_err[ix,3]
    de = q_calc_kj_err[ix,2]
    df = q_calc_kj_err[ix,2]

    bc = b*c 
    dbc = bc*(np.sqrt((db/b)**2 + (dc/c)**2))

    a_bc = a - bc 
    da_bc = np.sqrt((da)**2)+((dbc)**2)

    ###################################
    ef = e*f 
    Def = ef*(np.sqrt((de/e)**2 + (df/f)**2))
    
    d_ef = d - ef
    dd_ef = np.sqrt(dd**2 + Def**2)

    ########################################

    tot = a_bc/d_ef
    dtot = tot*(np.sqrt((da_bc/a_bc)**2)+ (dd_ef/d_ef)**2)

    tot_avo = tot*avogadro
    dtot_avo = tot_avo*(np.sqrt((tot/dtot)**2) + 0)

    tot_avo_RT = R*T - tot_avo
    dtot_avo_RT = np.sqrt(dtot_avo**2)
    
    all_tot.append(tot_avo_RT)
    all_dtot.append(abs(dtot))

print(press)
print(all_tot)
print(all_dtot)

### Converting to kg

In [ ]:
system_gram = 28914.48
unit_cell_gram = 803.18 # So this should be our gram/mol 

hads_kg = [] 
dhads_kg = [ ]
for ix in range(len(press)):
    a = all_tot[ix]/unit_cell_gram
    a = a*1000

    hads_kg.append(a)

    b = all_dtot[ix]/unit_cell_gram
    b = b*1000
    dhads_kg.append(b)

plt.rcParams['figure.figsize'] = [10, 8]


plt.title("Heat of Adsorption vs Pressure", fontsize=25)
plt.xlabel("Pressure (MPa)", fontsize=15)
plt.ylabel("Heat of Adsorption (kJ/kg)", fontsize=15)

color = iter(cm.rainbow(np.linspace(0, 1, len(press))))

for ix in range(len(press)):
    c = next(color)
    plt.errorbar(mpa[ix], hads_kg[ix], dhads_kg[ix], capsize=5, markersize=5, marker="o",fmt="none", color=c)
    plt.scatter(mpa[ix], hads_kg[ix], marker="o", color=c)

plt.xscale('log')

plt.show()

### Calculating Entropy of Adsorptio 

In [ ]:
sads_jmol = []
dsads_jmol = []

sads_kjmol = []
dads_kjmol = []
# Propagating as we would
   
for ix in range(len(press)):
    a = all_tot[ix]
    b = T

    da = all_dtot[ix]
    db = 0

    a_by_b = a/b
    da_by_b = a_by_b*(np.sqrt((db/b)**2 + (da/a)**2))
    
    sads_kjmol.append(a_by_b)
    dads_kjmol.append(da_by_b)

    a_by_b_j = a_by_b*1000

    da_by_b_j = a_by_b_j*(np.sqrt((da_by_b/a_by_b)**2))

    
    sads_jmol.append(a_by_b_j)
    dsads_jmol.append(da_by_b_j)

ambient_index = 10 
last_steps = 100
diff_n_unit = [] 
ddiff_n_unit = [] 
diff_press = [] 

for ix in range(len(press)):
    a1 = np.average(na[10][-last_steps:]) 
    a2 = np.average(na[ix][-last_steps:])

    da1 = np.std(na[10][-last_steps:]) 
    da2 = np.std(na[ix][-last_steps:])

    a_all = a1 - a2

    da_all = np.sqrt(da1**2 + da2**2)

    a = a_all/36
    da = da_all/36

    b = press[10] - press[ix]
    b = b/1000000
    diff_n_unit.append(a)
    ddiff_n_unit.append(da)
    diff_press.append(b)

### Matching S and N

In [ ]:
print("delta(P) in bar, Entropy change (J/K/kg)")

s_tot_mol = [] 
ds_tot_mol = [] 

s_tot_kg = [] 
ds_tot_kg = [] 

for ix in range(len(diff_press)):
    a = sads_jmol[ix]
    b = diff_n_unit[ix]

    da = dsads_jmol[ix]
    db = ddiff_n_unit[ix]

    ab = a*b
    dab = ab*(np.sqrt((da/a)**2) + (db/b)**2)

    s_tot_mol.append(ab)
    ds_tot_mol.append(abs(dab))

    s_tot_kg.append(1000*(ab/unit_cell_gram))
    ds_tot_kg.append(abs(1000*(dab/unit_cell_gram)))

    print(diff_press[ix], 1000*(ab/unit_cell_gram))


s_tot_mol = [] 
ds_tot_mol = [] 

s_tot_kg = [] 
ds_tot_kg = [] 

for ix in range(len(diff_press)):
    a = sads_jmol[10]
    b = diff_n_unit[ix]

    da = dsads_jmol[ix]
    db = ddiff_n_unit[ix]

    ab = a*b
    dab = ab*(np.sqrt((da/a)**2) + (db/b)**2)

    s_tot_mol.append(ab)
    ds_tot_mol.append(abs(dab))

    s_tot_kg.append(1000*(ab/unit_cell_gram))
    ds_tot_kg.append(abs(1000*(dab/unit_cell_gram)))

    print(diff_press[ix], 1000*(ab/unit_cell_gram))

In [ ]:
from scipy.integrate import trapezoid


def get_line(slope, intercept, steps):
    """Plot a line from slope and intercept"""
    x_vals = np.linspace(press[init_p], press[fin_press], steps)
    y_vals = intercept + slope * x_vals
    return(x_vals, y_vals)


init = 10 

diff_press_bar = []
stot = []  

for ix in range(nsteps-1):
    diff_x_na = (press[ix+1] - press[ix])
    a = diff_x_na/100000
    diff_press_bar.append(a)

    fin_press = ix+1
    init_p = ix

    diff_y_sads = sads_jmol[fin_press] - sads_jmol[init_p]
    ddiff_y_sads = np.sqrt(dsads_jmol[fin_press]**2 + dsads_jmol[init_p]**2)

    diff_x_sads = press[fin_press] - press[init_p]
    ddiff_x_sads = 0 

    m_sads = diff_y_sads/diff_x_sads # j/mol/K/Pa
    dm_sads = m_sads*(np.sqrt((ddiff_x_sads/diff_y_sads)**2 + (ddiff_y_sads/diff_y_sads)**2))

    #taking initial for the y-intercept 
    mx_sads = m_sads*press[init_p]
    dmx_sads = mx_sads*(np.sqrt((dm_sads/m_sads)**2 + (0/press[init_p])**2))

    c_sads =  sads_jmol[init_p] - mx_sads
    dc_sads = np.sqrt(dsads_jmol[init_p]**2 + dmx_sads**2)


    diff_y_na = np.average(na[fin_press][-100:])/36 - np.average(na[init_p][-100:])/36
    ddiff_y_na = np.sqrt((np.std(na[fin_press][-100:])/36)**2 + (np.std(na[init_p][-100:])/36)**2)

    diff_x_na = press[fin_press] - press[init_p]
    ddiff_x_na = 0 

    m_na = diff_y_na/diff_x_na # j/mol/K/Pa
    dm_na = m_na*(np.sqrt((ddiff_y_na/diff_y_na)**2 + (ddiff_x_na/diff_x_na)**2))

    #taking point 10 for the y-intercept 
    mx_na = m_na*press[init_p]
    dmx_na = mx_na*(np.sqrt((dm_na/m_na)**2 + (0/press[init_p])**2))

    c_na =  np.average(na[init_p][-100:])/36 - mx_na
    dc_na = np.sqrt((np.average(na[init_p][-100:])/36)**2 + dmx_na**2)

    sads_x, sads_y = get_line(m_sads,c_sads,2000)
    na_x, na_y = get_line(m_na, c_na, 2000)

    q = trapezoid(sads_y, na_y, dx=1.0, axis=-1)

    stot.append(q)

In [ ]:
# from ambient = starting at index 10 

# going upwards in pressure

inish = 10 

print(diff_press_bar[inish], stot[inish])
amb_diff_p_up = [] 
amb_diff_s_up = [] 
accum_p_up = diff_press_bar[inish]
accum_s_up =  stot[inish]

amb_diff_p_up.append(accum_p_up)
amb_diff_s_up.append(accum_s_up)

for ix in np.arange(11,16,1):
    accum_p_up = accum_p_up + diff_press_bar[ix]
    accum_s_up = accum_s_up + stot[ix]
    amb_diff_p_up.append(accum_p_up)
    amb_diff_s_up.append(accum_s_up)


# from ambient = starting at index 10 
# Going downwards 

inish = 10 
a = np.arange(0,9,1)
a = a[::-1]

print(diff_press_bar[inish], stot[inish])
amb_diff_p_down = [] 
amb_diff_s_down = [] 

accum_p_down = 0
accum_s_down =  0

for ix in a:
    print(ix)
    accum_p_down = accum_p_down + diff_press_bar[ix]
    accum_s_down = accum_s_down + stot[ix]
    amb_diff_p_down.append(-accum_p_down)
    amb_diff_s_down.append(-accum_s_down)
    

amb_diff_s_up_kg = []
amb_diff_s_down_kg = []

for ix in range(len(amb_diff_s_up)):
    a = 1000*(amb_diff_s_up[ix]/unit_cell_gram)
    amb_diff_s_up_kg.append(a)

for ix in range(len(amb_diff_s_down)):
    a = 1000*(amb_diff_s_down[ix]/unit_cell_gram)
    amb_diff_s_down_kg.append(a)



In [ ]:
fileout = open("h2o_ds.dat",'w')

fileout.write("dP (Pa), dS (J/kg/K) \n")

for ix in range(len(amb_diff_s_down_kg)):

    fileout.write("%s \t" %(amb_diff_p_down[ix]))    
    fileout.write("%s \n" %(amb_diff_s_down_kg[ix]))   
    

for ix in range(len(amb_diff_s_up_kg)):

    fileout.write("%s \t" %(amb_diff_p_up[ix]))    
    fileout.write("%s \n" %(amb_diff_s_up_kg[ix]))   
    
fileout.close() 